In [452]:
import collections
import math
import re
import sys
import os
sys.path.append(os.path.abspath("../"))
import itertools
from matplotlib import pyplot as plt
import functools
import pyparsing
import pandas as pd
import numpy as np
import random

In [2]:
from utils import *
from aoc import *
from personal import SESSION

In [3]:
aoc = AOC(session=SESSION)

In [4]:
aoc.verify_session()

True

In [657]:
data = aoc.get_today_file().analyse().head().data

Local file found.
143 empty line(s) found. Analyse as multiline data.
===== HEAD (5) =====
['Tile 2903:', '..#..#....', '.....#...#', '..#...#.##', '#....#..#.', '#.....#..#', '........##', '........#.', '#..##.....', '##.......#', '##.....##.']
['Tile 3659:', '###....##.', '....#...#.', '..........', '......#...', '#.........', '###.#.#...', '#....#...#', '#..#..#...', '....#.....', '..#.##..##']
['Tile 3691:', '#..##...#.', '.........#', '##.#.....#', '.#........', '.......#..', '..........', '##...#....', '..........', '#...#..#..', '###.#.#.##']
['Tile 3391:', '.#####.##.', '#.....#.##', '####......', '...#.#..##', '.#.#.#....', '#.#......#', '#.......##', '.#...#...#', '#.##.....#', '...#..#.#.']
['Tile 1867:', '.##....#.#', '........#.', '#...#....#', '..........', '.........#', '.........#', '#.#.#.....', '#...#.#...', '....#..#.#', '.##.##.###']


In [658]:
# aoc.brut = """Tile 2311:
# ..##.#..#.
# ##..#.....
# #...##..#.
# ####.#...#
# ##.##.###.
# ##...#.###
# .#.#.#..##
# ..#....#..
# ###...#.#.
# ..###..###

# Tile 1951:
# #.##...##.
# #.####...#
# .....#..##
# #...######
# .##.#....#
# .###.#####
# ###.##.##.
# .###....#.
# ..#.#..#.#
# #...##.#..

# Tile 1171:
# ####...##.
# #..##.#..#
# ##.#..#.#.
# .###.####.
# ..###.####
# .##....##.
# .#...####.
# #.##.####.
# ####..#...
# .....##...

# Tile 1427:
# ###.##.#..
# .#..#.##..
# .#.##.#..#
# #.#.#.##.#
# ....#...##
# ...##..##.
# ...#.#####
# .#.####.#.
# ..#..###.#
# ..##.#..#.

# Tile 1489:
# ##.#.#....
# ..##...#..
# .##..##...
# ..#...#...
# #####...#.
# #..#.#.#.#
# ...#.#.#..
# ##.#...##.
# ..##.##.##
# ###.##.#..

# Tile 2473:
# #....####.
# #..#.##...
# #.##..#...
# ######.#.#
# .#...#.#.#
# .#########
# .###.#..#.
# ########.#
# ##...##.#.
# ..###.#.#.

# Tile 2971:
# ..#.#....#
# #...###...
# #.#.###...
# ##.##..#..
# .#####..##
# .#..####.#
# #..#.#..#.
# ..####.###
# ..#.#.###.
# ...#.#.#.#

# Tile 2729:
# ...#.#.#.#
# ####.#....
# ..#.#.....
# ....#..#.#
# .##..##.#.
# .#.####...
# ####.#.#..
# ##.####...
# ##..#.##..
# #.##...##.

# Tile 3079:
# #.#.#####.
# .#..######
# ..#.......
# ######....
# ####.#..#.
# .#...#.##.
# #.#####.##
# ..#.###...
# ..#.......
# ..#.###..."""

In [659]:
data = aoc.analyse().head().data

143 empty line(s) found. Analyse as multiline data.
===== HEAD (5) =====
['Tile 2903:', '..#..#....', '.....#...#', '..#...#.##', '#....#..#.', '#.....#..#', '........##', '........#.', '#..##.....', '##.......#', '##.....##.']
['Tile 3659:', '###....##.', '....#...#.', '..........', '......#...', '#.........', '###.#.#...', '#....#...#', '#..#..#...', '....#.....', '..#.##..##']
['Tile 3691:', '#..##...#.', '.........#', '##.#.....#', '.#........', '.......#..', '..........', '##...#....', '..........', '#...#..#..', '###.#.#.##']
['Tile 3391:', '.#####.##.', '#.....#.##', '####......', '...#.#..##', '.#.#.#....', '#.#......#', '#.......##', '.#...#...#', '#.##.....#', '...#..#.#.']
['Tile 1867:', '.##....#.#', '........#.', '#...#....#', '..........', '.........#', '.........#', '#.#.#.....', '#...#.#...', '....#..#.#', '.##.##.###']


In [660]:
len(data)

144

In [661]:
piece_size = len(data[0][1:])

In [662]:
def get_borders(a):
    a_borders = [a[0], a[-1], ''.join([v[0] for v in a]), ''.join([v[-1] for v in a])]
    for poss in a_borders.copy():
        a_borders.append(poss[::-1])
    return a_borders

In [663]:
def compare(a, b):
    ba = get_borders(a)
    bb = get_borders(b)
    # print(ba, bb)
    return bool(set(ba).intersection(bb))

In [664]:
matrice = collections.defaultdict(dict)
data_up = dict()
for d1 in data:
    t1, d1 = d1[0], d1[1:]
    t1 = t1.replace('Tile ', '').replace(':', '')
    data_up[t1] = d1
    for d2 in data:
        t2, d2 = d2[0], d2[1:]
        t2 = t2.replace('Tile ', '').replace(':', '')
        if t1 == t2:
            matrice[t1][t2] = False
        else:
            matrice[t1][t2] = compare(d1, d2)
        matrice[t2][t1] = matrice[t1][t2]

In [665]:
df = pd.DataFrame(matrice)
# df.replace(False, np.NaN)
df.head()

,2903,3659,3691,3391,1867,3259,2441,2477,3533,3877,...,3919,1237,2803,1559,3083,1549,2777,1621,1847,1759
2903,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3659,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
3691,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3391,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1867,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [666]:
res = collections.defaultdict(list)
for column in df:
    res[df[column].value_counts()[True]].append(column)

In [667]:
math.prod(map(int, res[2]))

19955159604613

In [668]:
res

defaultdict(list,
            {3: ['2903',
              '3491',
              '1063',
              '1999',
              '2341',
              '1873',
              '3167',
              '2939',
              '3527',
              '1019',
              '1181',
              '1069',
              '3779',
              '1823',
              '1619',
              '3583',
              '2203',
              '3539',
              '2791',
              '2917',
              '3079',
              '2699',
              '3863',
              '2273',
              '2713',
              '1811',
              '3803',
              '2083',
              '1201',
              '1091',
              '2999',
              '2393',
              '2543',
              '2207',
              '1327',
              '1877',
              '1777',
              '1973',
              '2803',
              '2777'],
             4: ['3659',
              '3691',
              '3391',
              '3259',
       

## P2

In [669]:
def get_adj(id_):
    return tuple(df[str(id_)][df[str(id_)]].keys())

In [670]:
res[2]

['1867', '2441', '2633', '1663']

In [671]:
size = int(len(data)**0.5)

In [672]:
def common_adj(a, b, fmap):
    return set(get_adj(a)).intersection(get_adj(b)) - set([v for l in fmap for v in l])

In [673]:
def get_inti_adj(a, fmap):
    return set(get_adj(a)) - set([v for l in fmap for v in l])

In [700]:
final_map = [[0]*size for i in range(size)]

In [701]:
final_map[0][0] = res[2][0]
final_map[1][0], final_map[0][1] = get_adj(final_map[0][0])
for i in range(3, size*2):
    j = i%2
    k = i//2
    print(j, k)
    if j == 1:
        # print(final_map[j][k-1], final_map[j-1][k])
        # print(common_adj(final_map[j][k-1], final_map[j-1][k], final_map))
        final_map[j][k] = list(common_adj(final_map[j][k-1], final_map[j-1][k], final_map))[0]
    else:
        final_map[j][k] = list(common_adj(final_map[j][k-1], final_map[j][k-1], final_map))[0]



1 1
0 2
1 2
0 3
1 3
0 4
1 4
0 5
1 5
0 6
1 6
0 7
1 7
0 8
1 8
0 9
1 9
0 10
1 10
0 11
1 11


In [704]:
pd.DataFrame(final_map)

,0,1,2,3,4,5,6,7,8,9,10,11
0,1867,2543,1019,3079,3583,1823,2791,1201,1327,1619,3167,2441
1,2083,1979,2383,2551,3691,3631,2677,1171,1381,1471,1493,1873
2,2999,2579,1033,2309,2693,3373,1459,3331,3449,3659,2029,1091
3,3491,2161,1223,3617,3907,2333,1321,2729,3917,1549,2837,2803
4,2917,1483,3919,1487,3613,1237,3607,3457,1987,2473,2141,3539
5,2777,2111,2477,2503,3637,3359,3299,3727,3889,3559,3253,2341
6,2713,1453,3547,3467,3257,3877,1997,1597,1759,1153,2137,1063
7,1811,3461,2269,2153,1559,2311,1723,3533,1621,3191,1433,1999
8,1181,2861,1669,3989,2131,2251,3739,2753,3323,2411,1847,2273
9,2207,1693,3259,1667,2221,2857,3571,1151,3821,3391,3229,3527


In [703]:
for i in range(1, size-1):
    for j in range(0, size):
        final_map[i+1][j] = list(get_inti_adj(final_map[i][j], final_map))[0]

In [677]:
maps = pd.DataFrame(final_map).applymap(lambda x: data_up[x])

In [678]:
def set_portion(m, sm, i, j):
    for k, l in enumerate(sm):
        for f, v in enumerate(l):
            m[i+k][j+f] = v

In [679]:
cmap = [['?']*piece_size*size for i in range(piece_size*size)]

In [680]:
set_portion(
    cmap,
    np.flip([list(el) for el in maps[0][0]], 0), # real
    # np.flip(np.flip([list(el) for el in maps[0][0]], 1).T), # exemple
    0,
    0
)

In [681]:
def random_move_map(m):
    if random.random() > 0.5:
        return np.flip(np.flip(m, 1).T)
    return np.flip(m, 1)

In [682]:
def display(m):
    for l in m:
        print(''.join(l))

In [683]:
for j in range(0, size):
    if j > 0:
        current_map = [list(l) for l in maps[0][j]]
        last_bottom = [cmap[j*piece_size-1][k] for k in range(piece_size)]
        while last_bottom != (current_top := [current_map[0][k] for k in range(piece_size)]):
            current_map = random_move_map(current_map)

        set_portion(cmap, current_map, j*piece_size, 0)
    
    for i in range(1, size):
        decalage = i*piece_size
        current_map = [list(l) for l in maps[i][j]]
        last_right = [cmap[j*piece_size+k][decalage-1] for k in range(piece_size)]
        while last_right != (current_left := [current_map[k][0] for k in range(piece_size)]):
            current_map = random_move_map(current_map)

        set_portion(cmap, current_map, j*piece_size, decalage)

In [684]:
# cmap = np.flip(np.flip(np.flip(cmap, 1).T), 1)

In [685]:
display(cmap)

.##.##.####.#.#.######......#...#.##..#......#.#.##.#.#....#####.#...##.#.#.#.###.##.####.....#..#.##.#.#.#.#..#....#.#.
....#..#.##.##...............###.#.#......#.....###.................####.#.##.#....#.....##......#.##.##........#.......
#...#.#...............###..............##.....#.###.##..#..........##........#.##......#........#..###.........#.......#
#.#.#.......#......##.......######............#..###.......##.#......##........##...#....###...#.............##.#.#...#.
.........##.......#..##.#....##...........#...#..##...#....##.....#..##....##..###.#.....##......#...#...#...##..#..#..#
.........##.#....#....#.............#..##....#...##..#.##..###..........#...#.#..#.#.....###..#...###........##.......#.
...................#####..##.............#....#............##.#..#...##......####.......#..##....#....#..#.........###..
#...#....##..#.#...##.#...#....#.......##........###....#.....#.#......##...#.###..#...........#.#.....##........#......
........#.......#...............

In [686]:
final_map = []
for i, l in enumerate(cmap):
    if i%10 == 0 or i%10 == 9:
        continue
    final_line = []
    final_map.append(final_line)
    for j, v in enumerate(l):
        if j%10 == 0 or j%10 == 9:
            continue
        final_line.append(v)

In [687]:
# while ''.join(final_map[0]) != '.####...#####..#...###..':
#    final_map = random_move_map(final_map)

In [688]:
display(final_map)

...#..#..##.............#.#.#....#.....#...............##.#.##.#..#...........#..##......#......
...#.#...........###.................#.#.##..#........##......#.......#......#..#.......#.......
.#.#.....#.............####..........#..#........#.................#....#...#............#.#...#
...............###.#.............#...#.....#.........#......##..#.#...........#.#...#.....#..#..
.........#....#..#...........#......#.....#.##..#........#...#.##.#.....#..#...#...............#
................###..##.........#....#...........#..#.........##.......###....#..#..#.......###.
...#......#.#....#...#..#...............#....#...#.#....##...#.#..#.........#.#...##......#.....
.......#.....#...........##.#..........#........#......#..#.#.#.................###.....##....#.
.......#...##....#......#..#.....#..#..............#.........#.#.##..#......#...###...#.#.......
....................#...........................#.......#..#..##..#...###.....##...#.#....#.....
.#.#.##......................#

In [689]:
for i, l in enumerate(final_map):
    for j, v in enumerate(l):
        if is_pattern(i, j, variante, sea_monster):
            print('FIND !')

In [690]:
sea_monster = ['..................#.', '#....##....##....###', '.#..#..#..#..#..#...']

In [691]:
def is_pattern(originX, originY, map_, pattern):
    for i, l in enumerate(pattern):
        for j, v in enumerate(l):
            if v != '#':
                continue
            try:
                if map_[originY+j][originX+i] != v:
                    return False
            except IndexError:
                return False
    return True

In [692]:
variante = final_map
for k in range(100):
    variante = random_move_map(variante)
    a = 0
    for i, l in enumerate(final_map):
        for j, v in enumerate(l):
            if is_pattern(j, i, variante, sea_monster):
                a += 1
                print(f'{k} - FIND ! ({a})')

4 - FIND ! (1)
4 - FIND ! (2)
4 - FIND ! (3)
4 - FIND ! (4)
4 - FIND ! (5)
4 - FIND ! (6)
4 - FIND ! (7)
4 - FIND ! (8)
4 - FIND ! (9)
4 - FIND ! (10)
4 - FIND ! (11)
4 - FIND ! (12)
4 - FIND ! (13)
4 - FIND ! (14)
4 - FIND ! (15)
4 - FIND ! (16)
4 - FIND ! (17)
4 - FIND ! (18)
4 - FIND ! (19)
4 - FIND ! (20)
4 - FIND ! (21)
6 - FIND ! (1)
6 - FIND ! (2)
6 - FIND ! (3)
6 - FIND ! (4)
6 - FIND ! (5)
6 - FIND ! (6)
6 - FIND ! (7)
6 - FIND ! (8)
6 - FIND ! (9)
6 - FIND ! (10)
6 - FIND ! (11)
6 - FIND ! (12)
6 - FIND ! (13)
6 - FIND ! (14)
6 - FIND ! (15)
6 - FIND ! (16)
6 - FIND ! (17)
6 - FIND ! (18)
6 - FIND ! (19)
6 - FIND ! (20)
6 - FIND ! (21)
8 - FIND ! (1)
8 - FIND ! (2)
8 - FIND ! (3)
8 - FIND ! (4)
8 - FIND ! (5)
8 - FIND ! (6)
8 - FIND ! (7)
8 - FIND ! (8)
8 - FIND ! (9)
8 - FIND ! (10)
8 - FIND ! (11)
8 - FIND ! (12)
8 - FIND ! (13)
8 - FIND ! (14)
8 - FIND ! (15)
8 - FIND ! (16)
8 - FIND ! (17)
8 - FIND ! (18)
8 - FIND ! (19)
8 - FIND ! (20)
8 - FIND ! (21)
20 - FIND ! (1)
20 

In [697]:
amount = 21# max of above

15

In [698]:
sum([l.count('#') for l in final_map]) - amount * sum([l.count('#') for l in sea_monster])

1639